In [1]:
import cv2
import math
import os
import decimal
import random

# CREATE TILES

In [2]:
dirname = './Images'
dirLabel = './Annotations'
files = [f[:-4] for f in os.listdir(dirname) if f[-4:].lower() == '.jpg'] # getting a list of all imageNames in folder


In [ ]:
filepath = '%s/%s.txt'%(dirLabel, 'COCO_train2018_dji_0289')
imgpath = '%s/%s.jpg'%(dirname, 'COCO_train2018_dji_0289')


<b>Annotation format</b>: class x_center y_center width height 

Box coordinates are normalized xywh format (from 0 - 1).


## SPLIT ANNOTATIONS
In each annotation file:
<ul>
    <li>New annotations are created by splitting annotations that are in more than one tile</li>

   <li>New annotations are saved in "./Cropped/Annotations"</li>
</ul>




#### HOW TO USE:
fp = 'path/to/annotations'

<i>createNewAnnotations(fp, type)</i>

### getTiles

<b>input</b>: coords: [minValue, maxValue]

<b>output</b>: Array of tiles annotation occures  ex:[1,2,3]

In [ ]:
def getTiles(coords):
    tiles = []
    for no in coords:
        if(0 <= no < 0.2 ):
            tiles.append(0)
        if(0.2 <= no < 0.4 ):
            tiles.append(1)
        if(0.4 <= no < 0.6 ):
            tiles.append(2)
        if(0.6 <= no < 0.8 ):
            tiles.append(3)
        if(0.8 <= no <= 1):
            tiles.append(4)
    #print('tiles: %s'%(tiles))
    return(range(tiles[0], tiles[1]+1))


### splitAnnotation
Splitting and saving annotations according to tiles

<b>Annotation format</b>: class x_center y_center width height

<b>input</b>: 
<ul>
    <li>annotation: [class, xCenter, yCenter, width, height]</li>
    <li>filename: original name</li>
    <li>type: 'val'/'train'</li>
</ul>
<b>output</b>: void - Annotations are added to <i>Cropped/data/labels/'type'</i>

In [1]:
def splitAnnotation(annotation, filename, typ):
    clas, xcenter, ycenter, width, height = annotation
    xmin = float(xcenter) - (0.5*float(width))
    xmax = float(xmin) + (float(width))
    ymin = float(ycenter) - (0.5*float(height))
    ymax = float(ymin) + (float(height))
    if(ymax>1):
        ymax =1
    if(xmax>1):
        xmax =1    
    
    file = open('Cropped/data/labels/%s/%s'%(typ, filename), 'a+')
    
    xs = getTiles([xmin, xmax]) #Tiles annotation covers in x-direction
    ys = getTiles([ymin, ymax]) #Tiles annotation covers in y-direction
    
    
    for x in xs:
        for y in ys:
            #Bbox tile:
            txmin = x*0.2
            txmax = txmin+0.2
            tymin = y*0.2
            tymax = tymin+0.2
            
            #Bbox annotation in tile
            nxmin = txmin
            nxmax = txmax
            nymin = tymin
            nymax = tymax
            
            if(txmin <= xmin <txmax):
                nxmin = xmin
            if(txmin <= xmax <txmax):
                nxmax = xmax
            if(tymin <= ymin <tymax):
                nymin = ymin
            if(tymin <= ymax <tymax):
                nymax = ymax
            
            #new annotaion - only 6 decimals
            xcenter = decimal.Decimal((nxmax+nxmin)/2)
            ycenter = decimal.Decimal((nymax+nymin)/2)
            width = decimal.Decimal(nxmax-nxmin)
            height = decimal.Decimal(nymax-nymin)
            newA = '%s %s %s %s %s'%(clas, round(xcenter,6), round(ycenter,6), round(width,6), round(height,6) )
            file.write(newA)
            file.write('\n')            
    file.close()

### deleteAnnotations
<b>input</b>: folderpath

<b>output</b>: void - all files in directory removed

In [ ]:
def deleteAnnotations(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

### editAnnotation
<b>input</b>: 
<ul>
    <li>filepath: path/to/file</li>
    <li>filename: original name</li>
    <li>type: 'val'/'train'</li>
</ul>

<b>output</b>: void - runs splitAnnotation on all lines in file

In [ ]:
def editAnnotation(filepath, filename, typ):
    with open(filepath) as fp:  
        line = fp.readline()
        
        cnt = 1
        while line:
            ord = line.split()
            splitAnnotation(ord, filename, typ)
            line = fp.readline()
            cnt += 1

### createNewAnnotations
loops through all files in directory and runs editAnnotation

In [ ]:
def createNewAnnotations(dirname, typ):
    #deleteAnnotations('./Cropped/annotations')
    files = [f[:] for f in os.listdir('%s/%s/'%(dirname,typ)) if f[-4:].lower() == '.txt']
    for f in files:
        editAnnotation('%s/%s/%s'%(dirname,typ,f), f, typ)
    print('New annotations are created')

fp = './data2/labels'

createNewAnnotations(fp, 'val')

## CREATE TILES
<img src="img/tiles2.png" style="height: 250px; float: left; padding-right: 10px;"/>

Crop image to 5x5 tiles

Original image size: 4000 x 2250

Tile size: 800 x 450

Saving new tiles: "tile_i_j" in folder croppedImages


[0,0]: upper left corner 

img_shape: [height, width]

#### SPLITTING TILES AND SAVING ANNOTATIONS
### getAnnotations
<b>input</b>: 
<ul>
    <li>xmin</li>
    <li>xmax</li>
    <li>ymin</li>
    <li>ymax</li>
    <li>labelpath: path/to/label</li>
</ul>

<b>output</b>:Array of annotations in boundingbox

In [ ]:
def getAnnotations(xmin, xmax, ymin, ymax, labelpath):
    annotations = []
    with open(labelpath, 'r') as fp:  
        line = fp.readline()
        cnt = 1
        while line:
            clas, xcenter, ycenter, width, height = line.split()
            
            if((float(xmin) < float(xcenter) < float(xmax)) and (float(ymin) < float(ycenter) < float(ymax))):
                newx = (float(xcenter)-float(xmin))/0.2
                newy = (float(ycenter)-float(ymin))/0.2
                newWidth = float(width)/0.2
                newHeight = float(height)/0.2 
                newAnnotation = '%s %s %s %s %s'%(clas, newx, newy, newWidth, newHeight)
                annotations.append(newAnnotation)
            line = fp.readline()
            cnt += 1
    return annotations

### cropImg
<b>input</b>: 
<ul>
    <li>imgPath: path/to/img</li>
    <li>labelPath: path/to/label</li>
    <li>filename: original name</li>
    <li>type: 'val'/'train'</li>
</ul>

<b>output</b>: void - Tiling image to 25 new images. If annotation exsists in tile, tile and annotation is saved


In [ ]:
def cropImg(imgpath, labelpath, filename, type):
    img = cv2.imread(imgpath) # 4000  ×  2250   600 x 337.5
    
    img_shape = img.shape
    tile_size =  (int(img_shape[1]*0.2), int(img_shape[0]*0.2)) # 800 x 450
    offset = (int(img_shape[1]*0.2), int(img_shape[0]*0.2))

    for i in range(int(math.ceil(img_shape[0]/(offset[1] * 1.0)))):
        for j in range(int(math.ceil(img_shape[1]/(offset[0] * 1.0)))):
            xmin = offset[1]*i
            xmaks = min(offset[1]*i+tile_size[1], img_shape[0])
            ymin = offset[0]*j
            ymaks = min(offset[0]*j+tile_size[0], img_shape[1])
            
            jmin = 0.2*j
            jmax = jmin + 0.2
            imin = 0.2*i
            imax = imin + 0.2 
            
            annotations = getAnnotations( jmin, jmax, imin, imax, labelpath)
            if(len(annotations)!=0):
                cropped_img = img[xmin:xmaks, ymin:ymaks]
                cv2.imwrite('./Cropped/data/images/%s/%s_tile_%s_%s.jpg'%(type, filename, str(i), str(j)), cropped_img)
                file = open('Cropped/data/labels/%s/%s_tile_%s_%s.txt'%(type, filename, str(i), str(j)), 'a+')
                for line in annotations:
                    file.write(line)
                    file.write('\n')  
                file.close()
    print('Annotations and Images tiled')
           

### splitImagesAndReformatLabels
<b>input</b>: 
<ul>
    <li>files: array of filenames</li>
    <li>type: 'val'/'train'</li>
</ul>

<b>output</b>: void - runs cropImg on all inputfiles


In [ ]:
def splitImagesAndReformatLabels(files, type):
    dirimages = './Images'
    dirannotations = './Cropped/annotations'
    
    for file in files: 
        imagepath = '%s/%s.jpg'%(dirimages,file) 
        labelpath = '%s/%s.txt'%(dirannotations,file)
        cropImg(imagepath, labelpath, file, type)
    

#### HOW TO USE
<ul>
    <li>Delete folders in data/annotations and data/images </li>
    <li>splitImagesAndReformatLabels(files, type)</li>
</ul>


splitImagesAndReformatLabels(val, 'val2')

## OTHER USEFUL FUNCTIONS

#### COPY REP
Copies the whole "custom" directory to a new directory called "data2"

In [ ]:
def copyRep(type):
    dirImages = '/home/nina/custom/images/%s2018'%(type)
    dirLabels = '/home/nina/custom/labels/%s2018'%(type)
    files = [f[:-4] for f in os.listdir(dirImages) if f[-4:].lower() == '.jpg']
    print(len(files))
    import shutil
    wd = os.getcwd()
    for f in files:
        shutil.copy(('%s/%s.jpg'%(dirImages, f)), ('%s/data2/images/%s'%(wd, type)))
        shutil.copy(('%s/%s.txt'%(dirLabels, f)), ('%s/data2/labels/%s'%(wd, type)))        

copyRep('train')

#### CREATELOCATIONFILE
creating a locationFile that is later going to be placed in /yolov3/data

In [ ]:
def createLocationFile(type, files):
    list_file = open('%sLocations.txt'%(type), 'w')
    for f in files:
        list_file.write('../dataset/custom/Cropped/data/images/%s/%s.jpg'%(type, f))
        list_file.write('\n')


dirImages = './Cropped/data/images/%s'%('val')

files = [f[:-4] for f in os.listdir(dirImages) if f[-4:].lower() == '.jpg'] # getting a list of all imageNames in folder
createLocationFile('val',files)



### INTERSECTION
<img src="img/image.png" style="height: 250px; float: left"/>
#### INPUT : 
rec1: bottom-left(0, 0), top-right(10, 8),

rec2: bottom-left(2, 3), top-right(7, 9)
#### OUTPUT: 
(2, 3) (7, 8) (2, 8) (7, 3)

https://www.geeksforgeeks.org/intersecting-rectangle-when-bottom-left-and-top-right-corners-of-two-rectangles-are-given/ 


In [ ]:
def FindPoints(x1, y1, x2, y2, x3, y3, x4, y4, imgName): 
    
    # gives bottom-left point 
    # of intersection rectangle 
    x5 = max(x1, x3) 
    y5 = max(y1, y3) 
  
    # gives top-right point 
    # of intersection rectangle 
    x6 = min(x2, x4) 
    y6 = min(y2, y4) 
  
    # no intersection 
    if (x5 > x6 or y5 > y6) : 
        #print("NO INTERSECTION")
        return
        
    #New annotation
    width = max(x5,x6)-min(x5,x6)
    height = max(y5,y6)-min(y5,y6)
    x_center = min(x5,x6) + 0.5*width
    y_center = min(y5,y6) + 0.5*height
    new_annotation = 'Annotation: %s %s %s %s'%(x_center, y_center, width, height)
    print(new_annotation)
    
    